## Optmisation use case
### Drinks Distribution Problem

In [1]:
# pip install pulp

In [2]:
import pandas as pd
from pulp import *

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creates a list of all the supply nodes
warehouses = ["A", "B"]

In [4]:
# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000, "B": 4000}

In [5]:
# Creates a dictionary for the number of units of demand for each demand node
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}

In [6]:
# Creates a list of all demand nodes
restaurants = ["1", "2", "3", "4", "5"]

In [7]:
# Creates a list of costs of each transportation path
costs = [  # restaurants
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
]

In [8]:
# The cost data is made into a dictionary
costs = makeDict([warehouses, restaurants], costs, 0)

### Initialize an instance of LpProblem to represent our model:

In [9]:
# Creates the 'prob' variable to contain the problem data
problem = LpProblem("Drinks Distribution Problem", LpMinimize)

### List of possibe routes

In [10]:
# Creates a list of tuples containing all the possible routes for transport
routes = [(w, b) for w in warehouses for b in restaurants]

### Define decision variables

In [11]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("route", (warehouses, restaurants), 0, None, LpInteger)

The variables created determine the amount shipped on each route. The variables are defined
as pulp.LpInteger therefore solutions must not ship fractional numbers of crates.

### Setting objective function:
The objective is to minimise the total shipping cost of the solution.


In [12]:
# The objective function is added to 'prob' first
problem += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in routes]),
    "Sum_of_Transporting_Costs",
)

Now we have the objective function added and the model defined.

These constraints ensure that amount shipped from each brewery is less than the supply available. The names given to the constraints will be preserved when an `.lp' file is created.

## Constraints:

### Supply constraint

In [13]:
# Supply maximum constraints are added to prob for each supply node (warehouse)
    
for w in warehouses:
    problem += (
        lpSum([vars[w][b] for b in restaurants])<=supply[w],\
        "Sum_of_Products_out_of_Warehouse_%s" % w
    )

These constraints ensure that amount shipped to each bar is greater than the demand of the
bar. These could also be equality constraints, depending on how the problem is modelled.

### Demand constraint

In [14]:
# Demand minimum constraints are added to prob for each demand node (bar)
restaurants_demand_constraint = {}
for b in restaurants:
    constraint = lpSum([vars[w][b] for w in warehouses])>=demand[b]
    problem += constraint, "Sum_of_Products_into_Restaurants_%s"%b
    restaurants_demand_constraint[b] = constraint

In [15]:
# The problem data is written to an .lp file
problem.writeLP("DrinksDistributionProblem.lp")
for demand in range(500, 601, 10):
    # reoptimise the problem by increasing demand at bar '1'
    # note the constant is stored as the LHS constant not the RHS of the constraint
    restaurants_demand_constraint['1'].constant = - demand
    #or alternatively,
    #prob.constraints["Sum_of_Products_into_Restaurants_1"].constant = - demand

    # The problem is solved using PuLP's choice of Solver
    print('Solve problem:', problem.solve())

    # The status of the solution 
    print("Status:", LpStatus[problem.status])

    # Each of the variables with it's resolved optimum value
    for v in problem.variables():
        print(v.name, "=", v.varValue)

    # The optimised objective function value 
    print("Total Cost of Transportation = ", value(problem.objective))

Solve problem: 1
Status: Optimal
route_A_1 = 300.0
route_A_2 = 0.0
route_A_3 = 0.0
route_A_4 = 0.0
route_A_5 = 700.0
route_B_1 = 200.0
route_B_2 = 900.0
route_B_3 = 1800.0
route_B_4 = 200.0
route_B_5 = 0.0
Total Cost of Transportation =  8600.0
Solve problem: 1
Status: Optimal
route_A_1 = 300.0
route_A_2 = 0.0
route_A_3 = 0.0
route_A_4 = 0.0
route_A_5 = 700.0
route_B_1 = 210.0
route_B_2 = 900.0
route_B_3 = 1800.0
route_B_4 = 200.0
route_B_5 = 0.0
Total Cost of Transportation =  8630.0
Solve problem: 1
Status: Optimal
route_A_1 = 300.0
route_A_2 = 0.0
route_A_3 = 0.0
route_A_4 = 0.0
route_A_5 = 700.0
route_B_1 = 220.0
route_B_2 = 900.0
route_B_3 = 1800.0
route_B_4 = 200.0
route_B_5 = 0.0
Total Cost of Transportation =  8660.0
Solve problem: 1
Status: Optimal
route_A_1 = 300.0
route_A_2 = 0.0
route_A_3 = 0.0
route_A_4 = 0.0
route_A_5 = 700.0
route_B_1 = 230.0
route_B_2 = 900.0
route_B_3 = 1800.0
route_B_4 = 200.0
route_B_5 = 0.0
Total Cost of Transportation =  8690.0
Solve problem: 1
Sta

In [16]:
for v in problem.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

route_A_1 = 300.0
route_A_5 = 700.0
route_B_1 = 300.0
route_B_2 = 900.0
route_B_3 = 1800.0
route_B_4 = 200.0


In [18]:
print('Total transportation cost:', value(problem.objective))

Total transportation cost: 8900.0
